# Mini-Tarea 1: Análisis de datos de preferencias musicales

La presente actividad debe ser realizada de forma individual. El formato de entregar es el archivo .ipynb con todas las celdas ejecutadas. Las secciones donde se planteen preguntas de forma explícita, deben ser respondida en celdas de texto, y no se aceptará solo el output de una celda de código como respuesta.

**Fecha de entrega:** martes 13 de octubre de 2020, 09:00 hrs.

**Nombre alumno:** Alvaro Gustavo Riquelme Ramírez


# Apache Hive

## Instalación de ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
exec(open('hadoop_colab_installer.py').read())

Active services:
2880 DataNode
2960 NodeManager
2801 NameNode
3011 JobHistoryServer
3140 Jps
2725 ResourceManager



## Descarga de dataset

In [3]:
!wget -q --show-progress http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz
!tar xzf lastfm-dataset-1K.tar.gz

lastfm-dataset-1K.t 100%[===================>] 641.58M  1.90MB/s    in 5m 49s  


## Actividad 0

In [5]:
# Inserte su código para copia de datos en HDFS y creación de tablas en metastore
!hdfs dfs -mkdir users
!hdfs dfs -mkdir reprod
!hdfs dfs -put /content/lastfm-dataset-1K/userid-profile.tsv users
!hdfs dfs -put /content/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv reprod
!hdfs dfs -ls users
!hdfs dfs -ls reprod
!hdfs dfs -ls /user/root/

Found 1 items
-rw-r--r--   1 root supergroup      37842 2023-04-11 12:37 users/userid-profile.tsv
Found 1 items
-rw-r--r--   1 root supergroup 2529193595 2023-04-11 12:38 reprod/userid-timestamp-artid-artname-traid-traname.tsv
Found 2 items
drwxr-xr-x   - root supergroup          0 2023-04-11 12:38 /user/root/reprod
drwxr-xr-x   - root supergroup          0 2023-04-11 12:37 /user/root/users


In [ ]:
#%%writefile tabla.sql
#CREATE TABLE usuarios (userid INT, gender STRING, age INT, country STRING, signup STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
#LOAD DATA INPATH 'users' INTO table usuarios;

#CREATE TABLE reproduccion (userid INT, `timestamp` STRING, `musicbrainz-artist-id` INT, `artist-name` STRING, `musicbrainz-track-id` INT, `track-name` STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t';
#LOAD DATA INPATH 'reprod' INTO table reproduccion;

In [6]:
%%writefile tabla.sql
CREATE EXTERNAL TABLE usuario (`#id` STRING, gender STRING, age INT, country STRING, signup STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
LOCATION '/user/root/users';

CREATE EXTERNAL TABLE reproduccion (`#id` STRING, `timestamp` STRING, `musicbrainz-artist-id` STRING, `artist-name` STRING, `musicbrainz-track-id` STRING, `track-name` STRING) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
LOCATION '/user/root/reprod';

Writing tabla.sql


In [7]:
!hive -f tabla.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
OK
Time taken: 12.037 seconds
OK
Time taken: 0.192 seconds


## Actividad 1

In [8]:
%%writefile populares.sql
select `artist-name`, count(*) as cont FROM reproduccion GROUP BY `artist-name` ORDER BY cont desc LIMIT 0,10;

Writing populares.sql


In [9]:
!hive -f populares.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20230411123928_d189c123-7d62-4b4d-bc2b-00c59f3233d1
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 10
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.

Responda la siguiente pregunta:

¿ Cuál es el artista más popular y el menos popular dele ranking top-10 ?

El artista más popular es Radiohead con 115209 de reproducciones y el menos popular es Elliot Smith con 50278 reproducciones

## Actividad 2

In [10]:
%%writefile genero.sql
SELECT usuario.gender ,count(*) as cont ,reproduccion.`artist-name`
FROM usuario FULL OUTER JOIN reproduccion ON usuario.`#id` = reproduccion.`#id`
WHERE reproduccion.`artist-name` ='Radiohead' AND usuario.gender IS NOT NULL
GROUP BY usuario.gender,reproduccion.`artist-name`;


Writing genero.sql


In [11]:
!hive -f genero.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20230411124406_2a4b6869-c01b-4915-a3c2-d29e5e35a90c
Total jobs = 1
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuál es la cantidad de hombres y mujeres que han escuchado alguna canción del artista más popular ?

La cantidad de hombres que han escuchado radiohead es 63784 y de mujeres es 43748

## Actividad 3

In [12]:
%%writefile años.sql
SELECT usuario.age,COUNT (*) as conte, reproduccion.`artist-name`
FROM usuario FULL OUTER JOIN reproduccion ON usuario.`#id` = reproduccion.`#id`
WHERE reproduccion.`artist-name` ='Radiohead' AND usuario.age IS NOT NULL
GROUP BY usuario.age, reproduccion.`artist-name` ORDER BY conte asc;

Writing años.sql


In [13]:
!hive -f años.sql

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]

Logging initialized using configuration in jar:file:/content/apache-hive-2.3.9-bin/lib/hive-common-2.3.9.jar!/hive-log4j2.properties Async: true
Query ID = root_20230411124819_b544550a-7bb4-4a26-afde-f3a876a96a52
Total jobs = 2
SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/content/apache-hive-2.3.9-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/content/hadoop-2.10.1/share/hadoop/common/lib/sl

Responda la siguiente pregunta:

¿ Cuántos usuarios de 35 años han escuchado al artista más popular ?
1543 usuarios de 35 año han escuchado a Radiohead